In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import util
import pfilter_multi as pfilter
from scipy.stats import norm

import matplotlib.pyplot as plt

np.set_printoptions(precision=4)
!pwd

/home/zpyang/git/visnet/src/visnet/notebooks


In [2]:
class Camera:
    def __init__(self, cam_param, cam_pos, cam_att):
        self.param = cam_param
        self.pos = cam_pos
        self.att = cam_att
        self.K = util.get_cam_in(self.param)
        self.R = util.R_model2cam @ util.so3_exp(self.att)
        self.P = util.get_cam_mat_lie(self.param, self.pos, self.att)
#         self.measurements = []
        
    def __repr__(self):
        return "param: {}, pos: {}, att: {}".format(self.param, self.pos, self.att)
        
    def _get_pixel_pos_hom(self, target_pos):
        return self.P @ util.cart2hom(target_pos)
    
    def _get_pixel_pos(self, target_pos):
        return util.hom2cart(self._get_pixel_pos_hom(target_pos))
    
    def add_pixel_noise(self, pixel_coord):
        return pixel_coord + np.random.normal(0, 20)
    
class CamNode(Camera):
    def _get_vec(self, pixel_pos):
        """
        Back projected ray from the camera to the target (approximate)
        """
        pixel_coord = self.add_pixel_noise(pixel_pos)
        K = self.K
        R = self.R
        Bp = np.linalg.inv(K @ R)
        vec = Bp @ util.cart2hom(pixel_pos).astype('int32') # cast to integer because actual pixels are discrete
        vec = vec/np.linalg.norm(vec)
        return vec
    
    def _get_distance(self, target_pos):
        """
        !!!Should not be used directly in estimation!!!
        """
        dist =  np.linalg.norm(target_pos-self.pos)
        return dist
    
    def get_measurement(self, target_pos):
        """
        Bearing measurements in pixel coordinate and range in meters 
        The input here is temporary until we have trained the neural net to actually identify targets
        """
        bearing = self._get_pixel_pos(target_pos)
        return bearing

cam_param = [642.0926, 642.0926, 1000.5, 1000.5,0]
# [x y z roll pitch yaw]
cam_poses = np.array([
    [20, 20, 12, 0, 0, -2.2],
    [20, -20, 12, 0, 0, 2.2],
    [-20, -20, 12, 0, 0, 0.7],
    [-20, 20, 12, 0, 0, -0.7],
])

cam_nodes = [CamNode(cam_param, poses[0:3], poses[3:6]) for poses in cam_poses]

In [3]:
def dynamics_d(x):
    """
    Discrete dynamics
    """
    dt = 1/10
    A = np.block([
        [np.eye(3), dt*np.eye(3)],
        [np.zeros((3,3)), np.eye(3)]
    ])
    w = np.block([np.zeros(3), np.random.normal(0, 1, 3)])
    x_1 = A @ x + w
    
    return x_1

def drone_traj_circle(t):
    x = 15*np.sin(2 * np.pi * 0.01 * t / 3)
    y = 10*np.cos(2 * np.pi * 0.01 * t / 3)
    z = 20
    return np.array([x,y,z])

def drone_traj_straight(t):
    max_dist = 30
    x = -15 + t * 0.1
    y = 0
    z = 20
    return np.array([x, y, z])

In [4]:
num_cam = 4
n = 500        # number of particles per step
d = 6          # states to estimate (pos & vel)
h = 2*num_cam  # states of observation (pixel coord * num of cams)

def observe_fn(x):
    """
    Parameters:
        x : all particles at a step
    Returns:
        observations/measurements from x using the camera projection matrix
    """
    msmt = []
    for xi in x:
        vec = []
        for node in cam_nodes:
            p1 = node.get_measurement(xi[0:3])
            p2 = node.get_measurement(xi[6:9])
            vec = np.hstack([vec,p1, p2])
        msmt.append(vec)
    return np.array(msmt)/2000.0

def dynamics_fn(x):
    """
    Parameters:
        x : all particles at a time step
    Returns:
        x1: propagated particles
    """
    
    dt = 1/10
    A = np.block([
        [np.eye(3), dt*np.eye(3)],
        [np.zeros((3,3)), np.eye(3)]
    ])
    w = np.block([np.zeros(3), np.random.normal(0, 1, 3)])
    n = x.shape[0]
    A_block = np.block([[A, np.zeros((6,6))], [np.zeros((6,6)), A]])
    w = np.zeros((12, n))
    w[3:6, :] = np.random.normal(0, 1, (3, n))
    w[9:12, :] = np.random.normal(0, 1, (3,n))
    x1 = A_block @ x.T + w

    return x1.T

def prior_fn(n):
    pos_x = np.random.uniform(-20, 20, (n,1))
    pos_y = np.random.normal(-20, 20, (n,1))
    pos_z = np.random.normal(10, 25, (n,1))
    vel_x = np.random.normal(0, 1, (n,1))
    vel_y = np.random.normal(0, 1, (n,1))
    vel_z = np.zeros((n,1))
    particles = np.hstack([pos_x, pos_y, pos_z, vel_x, vel_y, vel_z])
    particles = np.hstack([particles,particles])
#     print(particles.shape)
    return particles


# def vec2coord(vec, dist):
#     v = vec
#     d = dist
#     theta = np.arctan2(v[1], v[0])
#     phi = np.arctan2(v[2], v[0]+v[1])
    
#     z = d * np.sin(phi)
#     x = d * np.cos(phi) * np.cos(theta)
#     y = d * np.cos(phi) * np.sin(theta)
#     return np.array([x,y,z])


In [5]:
pf = pfilter.ParticleFilter(
    prior_fn=prior_fn,
    observe_fn=observe_fn,
    dynamics_fn=dynamics_fn,
    n_particles=500,
    resample_fn = pfilter.residual_resample,
    weight_fn=lambda x, y: pfilter.squared_error(x, y, sigma=1),
    n_eff_threshold=1
)
particles = []
state = []
state_est = []
n_eff = []
t_step = 3
clutter_threshold = 1

for t in range(t_step):
    target_pos_1 = drone_traj_circle(t)
    target_pos_2 = drone_traj_straight(t)
    z = []
    random = np.random.uniform(0,1)
    for node in cam_nodes:
        z = np.hstack([z, node.get_measurement(target_pos_1), node.get_measurement(target_pos_2)])
        
    pf.update(z/2000.0)    
    particles.append(pf.particles)
    state_est.append(pf.map_state)
    state.append(np.hstack([target_pos_1, target_pos_2]))
    n_eff.append(pf.n_eff)
#     print('h: ', pf.hypotheses)
#     print('z: ', z) 
#     print('min weights', np.min(pf.weight_fn(pf.hypotheses, z/1000)))
#     print(pf.weight_normalisation)
#     print(pf.weights)

particles = np.array(particles)
state = np.array(state)
state_est = np.array(state_est)
err1 = state[:,0:3] - state_est[:, 0:3]
err2 = state[:,3:6] - state_est[:, 6:9]
n_eff = np.array(n_eff)

hyp (500, 16)
(500,)


ValueError: operands could not be broadcast together with shapes (500,12) (500,) 

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.axes(projection="3d")
ax.plot3D(state[:,0], state[:,1], state[:,2], '+-', label='truth')
ax.plot3D(state_est[:,0], state_est[:,1], state_est[:,2], 'x--', label='est')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.set_ylim(-20,20)
ax.set_zlim(10, 25)
plt.legend()

plt.figure(figsize=(10,10))
ax = plt.axes(projection="3d")
ax.plot3D(state[:,3], state[:,4], state[:,5], '+-', label='truth')
ax.plot3D(state_est[:,6], state_est[:,7], state_est[:,8], 'x--', label='est')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.set_ylim(-20,20)
ax.set_zlim(10, 25)
plt.legend()

plt.figure()
plt.subplot(2,1,1)
plt.plot(err1)
plt.title('error plot')
plt.xlabel('# of frames')
plt.ylabel('error[m]')

plt.subplot(2,1,2)
plt.plot(err2)
plt.title('error plot')
plt.xlabel('# of frames')
plt.ylabel('error[m]')

In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
# Attaching 3D axis to the figure
fig = plt.figure(figsize=(10, 10))
ax = p3.Axes3D(fig)
ax.set_xlabel('x, m')
ax.set_ylabel('y, m')
ax.set_zlabel('z, m')

# create trajectory data
cloud_x = np.hstack([particles[0, :, 0], particles[0, :, 6]])
cloud_y = np.hstack([particles[0, :, 1], particles[0, :, 7]])
cloud_z = np.hstack([particles[0, :, 2], particles[0, :, 8]])
cloud = ax.plot3D(cloud_x, cloud_y, cloud_z, 'yo', markersize=3, alpha = 0.3, label='particles')[0]

est_x = np.array([state_est[0, 0], state_est[0, 6]])
est_y = np.array([state_est[0, 1], state_est[0, 7]])
est_z = np.array([state_est[0, 2], state_est[0, 8]])
est = ax.plot3D(est_x, est_y, est_z, 'rP', markersize=10, label='estimation')[0]

cam_pos = ax.plot3D(cam_poses[:,0], cam_poses[:,1], cam_poses[:,2], 's', label='cameras')[0]
plt.legend()
ax.view_init(60,-90)

# fps = 5  # frames per second
# data_period = 0.1  # data period, seconds
# data_length = dataLines[0].shape[1]
# duration = data_period * data_length
# frames = int(np.floor(duration * fps))
# step = data_length // frames

def update_lines(i, cloud, est):
    # NOTE: there is no .set_data() for 3 dim data...
    x = np.hstack([particles[i, :, 0], particles[i, :, 6]])
    y = np.hstack([particles[i, :, 1], particles[i, :, 7]])
    z = np.hstack([particles[i, :, 2], particles[i, :, 8]])
    cloud.set_data(x, y)
    cloud.set_3d_properties(z)

    est_x = np.array([state_est[i, 0], state_est[i, 6]])
    est_y = np.array([state_est[i, 1], state_est[i, 7]])
    est_z = np.array([state_est[i, 2], state_est[i, 8]])
    est.set_data(est_x, est_y)
    est.set_3d_properties(est_z)
    return cloud, est

border = 0.1
ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)

ani = FuncAnimation(
    fig, update_lines, t_step, fargs=(cloud, est),
    interval=int(1000 / 10), blit=False)
HTML(ani.to_html5_video())
# plt.close()